 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Prepare-the-data" data-toc-modified-id="Prepare-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare the data</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Theano-Implementation" data-toc-modified-id="Theano-Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Theano Implementation</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Train-for-a-single-day" data-toc-modified-id="Train-for-a-single-day-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Train for a single day</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-all-of-2016" data-toc-modified-id="Post-processing-for-all-of-2016-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Post processing for all of 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-2016---benchmark-for-later-models" data-toc-modified-id="Post-processing-for-2016---benchmark-for-later-models-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Post-processing for 2016 - benchmark for later models</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Keras-implementation" data-toc-modified-id="Keras-implementation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Keras implementation</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Predict-for-one-day" data-toc-modified-id="Predict-for-one-day-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Predict for one day</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-2016" data-toc-modified-id="Post-processing-for-2016-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Post-processing for 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li></ul></li></ul></div>

# EMOS Network

The goal of this notebooks is to build and test a network implementation of EMOS, once in pure theano and then in keras. First we will try to replicate the results of the standard global EMOS. 

The reference period is always the mean CRPS for all dates and stations in 2016.

In [59]:
# Imports
# Note that the cost function only works with the theano backend for keras
from importlib import reload
import emos_network_theano; reload(emos_network_theano)
from  emos_network_theano import EMOS_Network
from crps_loss import crps_cost_function
import utils; reload(utils)
from utils import *
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt

In [76]:
# Basic setup
DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
results_dir = '../results/'
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Prepare the data

The interpolated data set contains the observations and forecasts from the 50 members for each station and each day.

In [3]:
# Chose a random date to illustrate the algorithm
date_str = '2011-02-14'   # This is our standard date format

In [9]:
# Load training and test set
time_start = timeit.default_timer()
train_set, test_set = get_train_test_sets(DATA_DIR, predict_date=date_str,
                                          fclt=fclt, window_size=window_size)
time_stop = timeit.default_timer()
print('Time: %.2f s' % (time_stop - time_start))

train set contains 25 days
test set contains 1 days
Time: 5.05 s


In [7]:
# Preload dataset 
raw_data = load_raw_data(DATA_DIR)

In [8]:
# Check how long it takes with preloaded data
import timeit
time_start = timeit.default_timer()
train_set, test_set = get_train_test_sets(preloaded_data=raw_data, predict_date=date_str,
                                          fclt=fclt, window_size=window_size)
time_stop = timeit.default_timer()
print('Time: %.2f s' % (time_stop - time_start))

train set contains 25 days
test set contains 1 days
Time: 0.89 s


In [35]:
train_set.features.shape, train_set.targets.shape, train_set.date_strs.shape

((12619, 2), (12619,), (12619,))

In [36]:
test_set.features.shape, test_set.targets.shape, test_set.date_strs.shape

((503, 2), (503,), (503,))

Not up to date anymore

Now we convert this dataset to use as the input for the EMOS networks. For this we pick a forecast date, and return as the training data all previous days within the window size previous to the start of the forecast. The test data is simply the data for the chosen forecast date. The 50 member ensemble is summarized by the mean and the standard deviation. Additionally, we remove all data where the observations are missing. Finally, the inputs are scaled. For this we subtract the mean from the mean and divide both the mean and the standard deviation by their standard deviations.

These 1D arrays contain the data for all dates and stations.

## Theano Implementation

To start with we build the model in pure theano. The model is defined in a separate script `EMOS_network_theano.py`. The network uses a custom CRPS loss function which is defined in `crps_loss.py`.

The EMOS_Network class is build to work in a similar way to keras models. For the fitting we are using gradient descent. Since we are using the entire dataset for each update, it is not stochastic. An early stopping algorithm is built into the fitting function. It stops training if the average training CRPS of the last 5 steps is decreasing by less than a parameter delta. 

### Train for a single day

To illustrate how the model work we will use the data for our example day above and fit the model.

In [12]:
# Define some model parameters
lr = np.asarray(0.1, dtype='float32')   # The learning rate
early_stopping_delta = 1e-4   # How much the CRPS must improve before stopping
steps_max = 1000   # How many steps to fit at max

In [13]:
# Set up the theano model
model_theano = EMOS_Network()

In [14]:
# Split the features into means and standard deviation
train_set.feature_names

['t2m_fc_mean', 't2m_fc_std']

In [15]:
train_mean = train_set.features[:, 0]
train_std = train_set.features[:, 1]
test_mean = test_set.features[:, 0]
test_std = test_set.features[:, 1]

In [17]:
# Train the model for some steps
model_theano.fit(train_mean, train_std, train_set.targets, steps_max, 
                 (test_mean, test_std, test_set.targets), lr=lr, 
                 early_stopping_delta=early_stopping_delta)
# Output is the training CRPS and the test CRPS

(array(1.1442610225440049), array(0.7698749147054791))

### Post processing for all of 2016

To compare the network model with the standard EMOS we will run it from 1 January 2016 to 31 December 2016. When looping over the days we are not resetting the model weights for each day. This drastically reduces the training time with identical results.

In [51]:
# Get start and stop indices
date_str_start = '2016-01-01'
date_str_stop = '2016-01-02'

In [52]:
model_theano = EMOS_Network()

In [60]:
# This function loops over the days.
train_crps_list, valid_crps_list, results_df = loop_over_days(
    DATA_DIR,
    model_theano,
    date_str_start, date_str_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    verbose=0,
    model_type='EMOS_Network_theano')

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


In [62]:
results_df.to_csv('csv_test.csv')

In [36]:
# Let's see what the mean prediction CRPS is
np.mean(valid_crps_list)

1.3055654671411516

In [17]:
np.save('./results/EMOS_network_theano.npy', valid_crps_list)

The standard EMOS global score (in `standard_postprocessing/emos_global.R`) is 1.0654. So we are doing a little better even, but this might just be chance.

### Post-processing for 2016 - benchmark for later models

Now the same process but only for the year of 2016. We can use this as a benchmark for later models.

In [18]:
date_idx_start = return_date_idx(dates, 2016, 1, 1)
date_idx_stop = return_date_idx(dates, 2016, 12, 31)
date_idx_start, date_idx_stop

(3285, 3650)

In [19]:
train_crps_list, valid_crps_list = loop_over_days(
    model_theano,
    tobs_full, 
    tfc_full, 
    date_idx_start, date_idx_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    model_type='EMOS_Network_theano')

3300
3400
3500
3600
Time: 23.99 s


In [20]:
np.mean(train_crps_list), np.mean(valid_crps_list)

(0.98131095989952555, 0.99854569302246976)

So the global EMOS benchmark for 2016 is around 1.00. 

## Keras implementation

Now let's build the same model in keras. This will provide a good starting point to expand the model later on.

In [37]:
# import the keras modules
# Note that the cost function only works with the theano backend
import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [38]:
# Let's build the model with Keras' functional API
# This is quite a bit easier and shorter than in theano 
def build_EMOS_Network_keras():
    mean_in = Input(shape=(1,))
    std_in = Input(shape=(1,))
    mean_out = Dense(1, activation='linear')(mean_in)
    std_out = Dense(1, activation='linear')(std_in)
    x = keras.layers.concatenate([mean_out, std_out], axis=1)
    return Model(inputs=[mean_in, std_in], outputs=x)

In [39]:
model_keras = build_EMOS_Network_keras()

In [40]:
model_keras.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 1)             2           input_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             2           input_2[0][0]                    
___________________________________________________________________________________________

In [52]:
# Compile the model with SGD optimizer and our custom loss function
opt = SGD(lr=0.1)  
model_keras.compile(optimizer=opt, loss=crps_cost_function, 
                    metrics=[crps_cost_function])

### Predict for one day

In keras the early stopping algorithm works slightly differently. It stops training once the training loss hasn't decreased by an amount delta in a certain number of steps (patience).

In [53]:
# This way we have the gradient descent on the whole training set just as in theano
batch_size = tfc_mean_train.shape[0]   

In [54]:
model_keras.fit([tfc_mean_train, tfc_std_train], tobs_train, epochs=steps_max, batch_size=batch_size,
          validation_data=[[tfc_mean_test, tfc_std_test], tobs_test], verbose=0,
          callbacks=[EarlyStopping(monitor='loss', min_delta=early_stopping_delta,
                                  patience=2)])

Train on 12619 samples, validate on 503 samples
Epoch 1/1000
12619/12619 [==============================] - 0s - loss: 3.3206 - crps_cost_function: 3.3206 - val_loss: 1.6199 - val_crps_cost_function: 1.6199
Epoch 2/1000
12619/12619 [==============================] - 0s - loss: 3.2698 - crps_cost_function: 3.2698 - val_loss: 1.6149 - val_crps_cost_function: 1.6149
Epoch 3/1000
12619/12619 [==============================] - 0s - loss: 3.2210 - crps_cost_function: 3.2210 - val_loss: 1.6103 - val_crps_cost_function: 1.6103
Epoch 4/1000
12619/12619 [==============================] - 0s - loss: 3.1739 - crps_cost_function: 3.1739 - val_loss: 1.6058 - val_crps_cost_function: 1.6058
Epoch 5/1000
12619/12619 [==============================] - 0s - loss: 3.1285 - crps_cost_function: 3.1285 - val_loss: 1.6015 - val_crps_cost_function: 1.6015
Epoch 6/1000
12619/12619 [==============================] - 0s - loss: 3.0847 - crps_cost_function: 3.0847 - val_loss: 1.5973 - val_crps_cost_function: 1.597

12619/12619 [==============================] - 0s - loss: 1.2301 - crps_cost_function: 1.2301 - val_loss: 0.8882 - val_crps_cost_function: 0.8882
Epoch 103/1000
12619/12619 [==============================] - 0s - loss: 1.2256 - crps_cost_function: 1.2256 - val_loss: 0.8844 - val_crps_cost_function: 0.8844
Epoch 104/1000
12619/12619 [==============================] - 0s - loss: 1.2212 - crps_cost_function: 1.2212 - val_loss: 0.8806 - val_crps_cost_function: 0.8806
Epoch 105/1000
12619/12619 [==============================] - 0s - loss: 1.2171 - crps_cost_function: 1.2171 - val_loss: 0.8771 - val_crps_cost_function: 0.8771
Epoch 106/1000
12619/12619 [==============================] - 0s - loss: 1.2131 - crps_cost_function: 1.2131 - val_loss: 0.8736 - val_crps_cost_function: 0.8736
Epoch 107/1000
12619/12619 [==============================] - 0s - loss: 1.2093 - crps_cost_function: 1.2093 - val_loss: 0.8704 - val_crps_cost_function: 0.8704
Epoch 108/1000
12619/12619 [=====================

12619/12619 [==============================] - 0s - loss: 1.1369 - crps_cost_function: 1.1369 - val_loss: 0.8003 - val_crps_cost_function: 0.8003
Epoch 203/1000
12619/12619 [==============================] - 0s - loss: 1.1368 - crps_cost_function: 1.1368 - val_loss: 0.8002 - val_crps_cost_function: 0.8002
Epoch 204/1000
12619/12619 [==============================] - 0s - loss: 1.1367 - crps_cost_function: 1.1367 - val_loss: 0.8001 - val_crps_cost_function: 0.8001
Epoch 205/1000
12619/12619 [==============================] - 0s - loss: 1.1367 - crps_cost_function: 1.1367 - val_loss: 0.8000 - val_crps_cost_function: 0.8000
Epoch 206/1000
12619/12619 [==============================] - 0s - loss: 1.1366 - crps_cost_function: 1.1366 - val_loss: 0.8000 - val_crps_cost_function: 0.8000
Epoch 207/1000
12619/12619 [==============================] - 0s - loss: 1.1366 - crps_cost_function: 1.1366 - val_loss: 0.7999 - val_crps_cost_function: 0.7999
Epoch 208/1000
12619/12619 [=====================

In [ ]:
# Get train and test CRPS
(model_keras.evaluate([tfc_mean_train, tfc_std_train], tobs_train, batch_size, verbose=0), 
 model_keras.evaluate([tfc_mean_test, tfc_std_test], tobs_test, batch_size, verbose=0))

Here we see that the CRPS is slightly higher than in our theano implementation. This is most likely due to the differences in the early stopping algorithm.

### Post-processing for 2016

Same as above with the theano model.

In [55]:
date_idx_start = return_date_idx(dates, 2016, 1, 1)
date_idx_stop = return_date_idx(dates, 2016, 12, 31)

In [41]:
model_keras = build_EMOS_Network_keras()
opt = SGD(lr=0.1)  
model_keras.compile(optimizer=opt, loss=crps_cost_function, 
                    metrics=[crps_cost_function])

In [45]:
# This function loops over the days.
train_crps_list, valid_crps_list = loop_over_days(
    DATA_DIR,
    model_keras,
    date_str_start, date_str_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    verbose=0,
    model_type='EMOS_Network_keras')

100%|██████████| 31/31 [01:14<00:00,  2.76s/it]

Time: 74.96 s


So the keras implementation is quite a bit slower than the pure theano version. This could be due to the overhead of calling model.fit many many times.

In [46]:
np.mean(train_crps_list), np.mean(valid_crps_list)

(1.1908277614435003, 1.3074258575308064)

The results are very similar to the theano implementation.

### Train 2015, predict 2016

Finally, we will train one single model on all of the 2015 data, and then post-process all of 2016 with this one model. 

In [63]:
train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']

In [64]:
# Load data sets
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates)

train set contains 365 days
test set contains 366 days


In [65]:
model_keras = build_EMOS_Network_keras()
opt = Adam(lr=0.1)  # Adam is a better SGD in a nutshell
model_keras.compile(optimizer=opt, loss=crps_cost_function)

In [67]:
# Split dataset
train_mean = train_set.features[:, 0]
train_std = train_set.features[:, 1]
test_mean = test_set.features[:, 0]
test_std = test_set.features[:, 1]

In [68]:
model_keras.fit([train_mean, train_std], train_set.targets, epochs=10, 
                batch_size=1024, 
                validation_data=[[test_mean, test_std], test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 3.3732 - val_loss: 2.2165
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.5464 - val_loss: 1.1362
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.0852 - val_loss: 1.0150
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0126
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0111
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0113
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0116
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 1.0694 - val_loss: 1.0114
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0128
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 1.0

In [69]:
# Get predictions
preds = model_keras.predict([test_mean, test_std])

In [74]:
# Save predictions
results_df = create_results_df(test_set.date_strs, test_set.station_ids,
                               preds[:, 0], preds[:, 1])

In [75]:
results_df.head()

,date,mean,station_id,std
0,2016-01-01,4.442505,44.0,1.651832
1,2016-01-01,1.637181,71.0,2.386799
2,2016-01-01,0.611089,73.0,1.772206
3,2016-01-01,4.397912,78.0,1.700036
4,2016-01-01,1.966224,91.0,2.544652


In [77]:
results_df.to_csv(results_dir + 'emos_network_train_2015_pred_2016.csv')

In [27]:
tfc_mean_train, tfc_std_train, tobs_train, tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx_start, window_size=365, fclt=0, 
                            subtract_std_mean=False, test_plus=365)

In [28]:
tfc_mean_train.shape, tfc_mean_test.shape

((180849,), (181718,))

In [29]:
model_keras = build_EMOS_Network_keras()
opt = Adam(lr=0.1)  # Adam is a better SGD in a nutshell
model_keras.compile(optimizer=opt, loss=crps_cost_function)

In [30]:
model_keras.fit([tfc_mean_train, tfc_std_train], tobs_train, epochs=10, 
                batch_size=1024, 
                validation_data=[[tfc_mean_test, tfc_std_test], tobs_test])

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 2.3505 - val_loss: 1.0104
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0107
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0120
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0158
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 1.0694 - val_loss: 1.0093
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 1.0695 - val_loss: 1.0099
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 1.0694 - val_loss: 1.0098
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 1.0695 - val_loss: 1.0121
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 1.0697 - val_loss: 1.0108
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 1.0

So we get a 2016 CRPS of 1.01 compared to 1.00 for the 25 day rolling window. Surprisingly little difference. This suggests that the seasonality is not important.